In [26]:
import sys
sys.path.insert(1,'../Utilities')
import json
import requests
import time
from data import *
from trophics import *
from ecoNameManipulations import *
import pickle
import requests


In [27]:
eiggRawData = eiggRawData()
eiggRawData = eiggRawData["Scientific name"].str.lower()
eiggRawData = set(eiggRawData)
len(eiggRawData)

C:\Users\davie\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2862: DtypeWarning: Columns (9,22) have mixed types. Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


3177

In [28]:
def validateListedEiggSpecies(speciesList):
    validatedSpeciesResultDict = {}
    counter = 0
    for species in eiggRawData:
        if counter % 100 == 0: print("Step " + str(counter))
        indivResult = cleanSpeciesName(species)
        validatedSpeciesResultDict[species] = indivResult
        counter += 1
    
    print(len(validatedSpeciesResultDict))
    with open("PartialEiggVerified",'wb') as f:
        pickle.dump(validatedSpeciesResultDict,f)

In [29]:
validateListedEiggSpecies(eiggRawData)

Step 0
Step 100
Step 200
Step 300
Error occured at encrusting algae
Step 400
Step 500
Error occured at filamentous brown
Step 600
Error occured at platyhypnidium ripariodes
Step 700
Step 800
Step 900
Error occured at rubus sect.
Error occured at maerl
Error occured at tenellia caerulea
Step 1000
Error occured at leiocolea collaris
Error occured at porifera crusts
Error occured at rubus sect.
Step 1100
Step 1200
Step 1300
Step 1400
Step 1500
Error occured at lichenicolous fungus
Step 1600
Step 1700
Step 1800
Step 1900
Error occured at vertebrata fucoides
Step 2000
Error occured at leiocolea turbinata
Step 2100
Error occured at cirrhia icteritia
Step 2200
Error occured at leiocolea fitzgeraldiae
Step 2300
Step 2400
Error occured at diatoms colonial
Error occured at diatoms film
Step 2500
Step 2600
Step 2700
Step 2800
Step 2900
Step 3000
Error occured at bryozoa crusts
Step 3100
3177


# How do we deal with the failed entries?

Some are genuinely incorrect- some on the other hand the API itself didnt catch,
so I will manually add in the ones that are valid

## I will fix the rest manually

In [ ]:
error occured at tenellia caerulea YEP
error occured at leiocolea turbinata YEP
error occured at vertebrata fucoides
error occured at diatoms film
error occured at porifera indet
error occured at aphodius fimetarius/pedellus
error occured at leiocolea fitzgeraldiae
error occured at rubus sect.
error occured at cirrhia icteritia
error occured at platyhypnidium ripariodes
error occured at diatoms colonial
error occured at rubus sect.
error occured at anthus spinoletta/petrosus
error occured at filamentous brown (algae)
error occured at leiocolea collaris

In [ ]:
amendedDict = {}
amendedDict['bryozoa crusts'] = 'bryozoa'
amendedDict['rubus sect.'] = 'rubus'
amendedDict['leiocolea collaris'] = 'leiocolea collaris'
amendedDict['leiocolea fitzgeraldiae'] = 'leiocolea fitzgeraldiae'
amendedDict['platyhypnidium ripariodes'] = 'platyhypnidium ripariodes'
amendedDict['vertebrata fucoides'] = 'vertebrata fucoides'
amendedDict['porifera crusts'] = 'porifera'
amendedDict['diatoms brown'] = 'diatoms'
amendedDict['diatoms colonial'] = 'diatoms'
amendedDict['cirrhia icteritia'] = 'cirrhia icteritia'
amendedDict['leiocolea turbinata'] = 'leiocolea turbinata'
amendedDict['maerl'] = 'maerl'
amendedDict['tenellia caerulea'] = 'tenellia caerulea'

In [ ]:
fullSpeciesOfEiggDict = {}

with open("PartialEiggVerified",'rb') as f:
    partialDict = pickle.load(f)

for item in partialDict:
    if len(partialDict[item]) > 0:
        fullSpeciesOfEiggDict[item] = partialDict[item]
    
for item in amendedDict:
    fullSpeciesOfEiggDict[item] = amendedDict[item]
    
with open("EiggVerifiedSpeciesList",'wb') as f:
    pickle.dump(fullSpeciesOfEiggDict,f)


In [ ]:
fullSpeciesOfEiggDict['carex hostiana x viridula = c. x fulva']

In [ ]:
len(fullSpeciesOfEiggDict)

In [ ]:
with open("EiggVerifiedSpeciesList",'rb') as f:
    testingPickleWrite = pickle.load(f)
    
print(len(testingPickleWrite))

# Conclusion; missing 4 species (X crusts + X film)

# Family mismatches

In [ ]:
def rawObsData(filename):
    data = pd.read_csv(filename) 
    df = data[['Scientific name', 'Family']]

    df = df.dropna(subset=['Scientific name'])
    df = df.dropna(subset=['Family'])

    return df

def familyVerification(speciesName):
    callToValidateName = requests.get('http://resolver.globalnames.org/name_resolvers.json?names='+name)
    jsonRes = callToValidateName.json()['data'][0]
    print(jsonRes)

def apiFire(name):
    callToValidateName = requests.get('http://resolver.globalnames.org/name_resolvers.json?names='+name)
    jsonRes = callToValidateName.json()['data'][0]['results'][0]
    groups,values = jsonRes['classification_path_ranks'], jsonRes['classification_path']
    groups = groups.lower().split("|") 
    # ['', 'superkingdom', 'kingdom', 'phylum', 'subphylum', '', '', '', '', '', '', '', '', '', '', '', 'order', 'family', 'subfamily', '', 'genus', 'species']
    values = values.lower().split("|")

    taxonomy = dict(zip(groups,values))

    if 'family' in taxonomy:
        return taxonomy['family'].lower().strip()
    return ""
    
islandDatasets = "C:/Users/davie/Desktop/Masters/Dissertation/Code/DissertationCode/Eigg/IslandDatasets/"
df = rawObsData(islandDatasets+"eigg.csv")
for index, row in df.iterrows():
    try:
        if not (row['Family'].lower().strip() == apiFire(row['Scientific name'])):
            print(row['Scientific name'], row['Family'])
    except:
        pass

C:\Users\davie\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2802: DtypeWarning: Columns (9,22) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


Helophorus brevipalpis Hydrophilidae
Helophorus brevipalpis Hydrophilidae
Gerris (Gerris) costae Gerridae
Acicula fusca Aciculidae
Euphrasia arctica x confusa Orobanchaceae
Euphrasia arctica x confusa Orobanchaceae
Euphrasia arctica x confusa Orobanchaceae
Euphrasia arctica x confusa Orobanchaceae
Sigara (Retrocorixa) venusta Corixidae
Sigara (Subsigara) falleni Corixidae
Corixa iberica Corixidae
Sigara (Retrocorixa) venusta Corixidae
Sigara (Subsigara) distincta Corixidae
Sigara (Subsigara) scotti Corixidae
Silpha atrata Silphidae
Chara virgata var. virgata Characeae
Hygrocybe acutoconica var. acutoconica Hygrophoraceae
Exidia nucleata Auriculariaceae
Exidia saccharina Auriculariaceae
Anthyllis vulneraria vulneraria langei Fabaceae
Bromus hordeaceus subsp. thominei Poaceae
Hieracium ebudicum Asteraceae
Rorippa nasturtium-aquaticum Brassicaceae
Arthrinium fuckelii Apiosporaceae
Lecidella elaeochroma f. soralifera Lecanoraceae
Ixodes (Ixodes) ricinus Ixodidae
Ixodes (Ixodes) ricinus Ixo

Crataegus monogyna Rosaceae
Blechnum spicant Blechnaceae
Trichophorum germanicum Cyperaceae
Ranunculus ficaria Ranunculaceae
Senecio jacobaea Asteraceae
Chamerion angustifolium Onagraceae
Galium palustre subsp. palustre Rubiaceae
Rubus sect. Rubus Rosaceae
Hieracium duriceps Asteraceae
Hieracium shoolbredii Asteraceae
Hieracium duriceps Asteraceae
Senecio aquaticus Asteraceae
Triglochin palustre Juncaginaceae
Triglochin palustre Juncaginaceae
Salix cinerea subsp. oleifolia Salicaceae
Potentilla palustris Rosaceae
Ranunculus ficaria Ranunculaceae
Hieracium langwellense Asteraceae
Hieracium shoolbredii Asteraceae
Hieracium duriceps Asteraceae
Senecio aquaticus Asteraceae
Ranunculus ficaria Ranunculaceae
Hieracium shoolbredii Asteraceae
Lasiocampa quercus callunae Lasiocampidae
Abraxas grossulariata Geometridae
Galium palustre subsp. palustre Rubiaceae
Atriplex praecox Amaranthaceae
Rorippa microphylla Brassicaceae
Bromopsis ramosa Poaceae
Senecio jacobaea Asteraceae
Carex viridula subsp.

Munida rugosa Munididae
Necora puber Polybiidae
Necora puber Polybiidae
Necora puber Polybiidae
Lacuna vincta Littorinidae
Pawsonia saxicola Cucumariidae
Necora puber Polybiidae
Necora puber Polybiidae
Necora puber Polybiidae
Ciona intestinalis Cionidae
Necora puber Polybiidae
Pawsonia saxicola Cucumariidae
Suberites carnosus Suberitidae
Necora puber Polybiidae
Ciona intestinalis Cionidae
Necora puber Polybiidae
Steromphala cineraria Trochidae
Steromphala cineraria Trochidae
Ciona intestinalis Cionidae
Munida rugosa Munididae
Necora puber Polybiidae
Scinaia furcellata Scinaiaceae
Steromphala cineraria Trochidae
Steromphala cineraria Trochidae
Suberites carnosus Suberitidae
Ciona intestinalis Cionidae
Steromphala cineraria Trochidae
Luidia sarsii Luidiidae
Porella compressa Bryocryptellidae
Necora puber Polybiidae
Necora puber Polybiidae
Carcinus maenas Portunidae
Porella compressa Bryocryptellidae
Ciona intestinalis Cionidae
Pawsonia saxicola Cucumariidae
Saccorhiza polyschides Phyllar

Mycomicrothelia confusa Trypetheliaceae
Verrucaria fusconigrescens Verrucariaceae
Anisomeridium ranunculosporum Monoblastiaceae
Lecanora dispersa Lecanoraceae
Opegrapha gyrocarpa Roccellaceae
Pyrenidium actinellum Dacampiaceae
Opegrapha atra Roccellaceae
Varicellaria lactea Ochrolechiaceae
Lecanora helicopis Lecanoraceae
Dirina massiliensis f. sorediata Roccellaceae
Arthopyrenia carneobrunneola Arthopyreniaceae
Dimerella lutea Coenogoniaceae
Caloplaca marina Teloschistaceae
Brianaria bauschiana Psoraceae
Bacidia trachona Ramalinaceae
Aspicilia grisea Megasporaceae
Opegrapha calcarea Roccellaceae
Stigmidium microspilum Mycosphaerellaceae
Leptogium burgessii Collemataceae
Lecidella elaeochroma f. soralifera Lecanoraceae
Aspicilia recedens Megasporaceae
Porina borreri Porinaceae
Amandinea pelidna Caliciaceae
Haematomma ochroleucum var. porphyrium Haematommataceae
Arthopyrenia nitescens Arthopyreniaceae
Opegrapha varia Roccellaceae
Dimerella lutea Coenogoniaceae
Arthonia stellaris Arthonia

...

Hedwigia integrifolia Hedwigiaceae
Salsola kali subsp. kali Amaranthaceae
Abraxas grossulariata Geometridae
Abraxas grossulariata Geometridae
Notiophilus aesthuans Carabidae
Coeloglossum viride Orchidaceae
Bibio pomonae Bibionidae
Hieracium subcrocatum Asteraceae
Silpha atrata Silphidae
Lecidella elaeochroma f. soralifera Lecanoraceae
Lecanora helicopis Lecanoraceae
Arthopyrenia punctiformis Arthopyreniaceae
Lecanora actophila Lecanoraceae
Sterna hirundo Sternidae
Hieracium strictiforme Asteraceae
Arthonia cinnabarina Arthoniaceae
Sterna hirundo Sternidae
Pertusaria pseudocorallina Pertusariaceae
Aglais io Nymphalidae
Arthopyrenia carneobrunneola Arthopyreniaceae
Polychidium muscicola Placynthiaceae
Mycoblastus caesius Mycoblastaceae
Bactrospora homalotropa Roccellaceae
Micarea botryoides Pilocarpaceae
Strangospora moriformis Strangosporaceae
Arthonia elegans Arthoniaceae
Pertusaria pseudocorallina Pertusariaceae
Gyalecta derivata Gyalectaceae
Bactrospora homalotropa Roccellaceae
Lobar

Tetrix undulata Tetrigidae
Haematopus ostralegus Charadriidae
Pseudorchis albida Orchidaceae
Lecanora dispersa Lecanoraceae
Aphanolejeunea microscopica Lejeuneaceae
Regulus regulus Regulidae
Plagiothecium undulatum Plagiotheciaceae
Turdus viscivorus Muscicapidae
Troglodytes troglodytes Troglodytidae
Turdus philomelos Muscicapidae
Pseudorchis albida Orchidaceae
Senecio aquaticus Asteraceae
Hieracium duriceps Asteraceae
Regulus regulus Regulidae
Mycoblastus caesius Mycoblastaceae
Cololejeunea minutissima Lejeuneaceae
Erithacus rubecula Muscicapidae
Calliergonella cuspidata Hypnaceae
Hypoxylon fuscum Xylariaceae
Turdus merula Muscicapidae
Turdus viscivorus Muscicapidae
Taraxacum subnaevosum Asteraceae
Cornu aspersum Helicidae
Agrocybe pediades Strophariaceae
Plagiomnium undulatum Plagiomniaceae
Bilimbia sabuletorum Lecideaceae
Taraxacum faeroense Asteraceae
Isothecium myosuroides var. myosuroides Lembophyllaceae
Atriplex laciniata Amaranthaceae
Regulus regulus Regulidae
Psathyrella obtusa

Carabus clatratus Carabidae
Lejeunea patens Lejeuneaceae
Regulus regulus Regulidae
Vitrea crystallina Pristilomatidae
Rosa caesia subsp. vosagiaca Rosaceae
Carychium minimum Carychiidae
Notiophilus aesthuans Carabidae
Lichenomphalia alpina Hygrophoraceae
Taraxacum subnaevosum Asteraceae
Micarea byssacea Pilocarpaceae
Rorippa nasturtium-aquaticum Brassicaceae
Arthothelium lirellans Arthoniaceae
Cololejeunea minutissima Lejeuneaceae
Erithacus rubecula Muscicapidae
Nardia scalaris Jungermanniaceae
Isothecium myosuroides var. myosuroides Lembophyllaceae
Zygodon viridissimus var. viridissimus Orthotrichaceae
Microtus agrestis Muridae
Psilocybe coprophila Strophariaceae
Turdus philomelos Muscicapidae
Pseudorchis albida Orchidaceae
Carex viridula subsp. oedocarpa Cyperaceae
Turdus merula Muscicapidae
Rubus venetorum Rosaceae
Elytrigia juncea Poaceae
Lagopus lagopus subsp. scotica Phasianidae
Helictotrichon pubescens Poaceae
Anastrophyllum minutum Scapaniaceae
Turdus merula Muscicapidae
Anastr

Blysmus rufus Cyperaceae
Lasallia pustulata Umbilicariaceae
Racomitrium aquaticum Grimmiaceae
Phalacrocorax aristotelis Phalacrocoracidae
Troglodytes troglodytes Troglodytidae
Plagiomnium ellipticum Plagiomniaceae
Hieracium vennicontium Asteraceae
Aegopinella pura Oxychilidae
Turdus merula Muscicapidae
Dirina massiliensis f. sorediata Roccellaceae
Turdus viscivorus Muscicapidae
Punctum (Punctum) pygmaeum Punctidae
Haematopus ostralegus Charadriidae
Troglodytes troglodytes Troglodytidae
Bacidia trachona Ramalinaceae
Rhytidiadelphus triquetrus Hylocomiaceae
Taraxacum faeroense Asteraceae
Mycena chlorantha Mycenaceae
Regulus regulus Regulidae
Racomitrium elongatum Grimmiaceae
Rhizomnium punctatum Cinclidiaceae
Taraxacum maculosum Asteraceae
Pseudorchis albida Orchidaceae
Emberiza citrinella Emberizidae
Troglodytes troglodytes Troglodytidae
Pseudorchis albida Orchidaceae
Galerina hypnorum Cortinariaceae
Turdus viscivorus Muscicapidae
Cotoneaster integrifolius Rosaceae
Vertigo (Vertigo) sub

Regulus regulus Regulidae
Corvus cornix subsp. cornix Corvidae
Racomitrium fasciculare Grimmiaceae
Haematopus ostralegus Charadriidae
Verrucaria fusconigrescens Verrucariaceae
Agrocybe paludosa Strophariaceae
Aphanolejeunea microscopica Lejeuneaceae
Dimerella lutea Coenogoniaceae
Regulus regulus Regulidae
Pterogonium gracile Leucodontaceae
Coeloglossum viride Orchidaceae
Turdus merula Muscicapidae
Exidia nucleata Auriculariaceae
Turdus viscivorus Muscicapidae
Emberiza schoeniclus Emberizidae
Zygodon viridissimus var. viridissimus Orthotrichaceae
Helictotrichon pubescens Poaceae
Arthopyrenia cerasi Arthopyreniaceae
Thymus polytrichus subsp. britannicus Lamiaceae
Taraxacum stictophyllum Asteraceae
Pseudorchis albida Orchidaceae
Emberiza citrinella Emberizidae
Dimerella pineti Coenogoniaceae
Corvus cornix subsp. cornix Corvidae
Haematopus ostralegus Charadriidae
Lachnum corticale Lachnaceae
Helicella itala Hygromiidae
Haematopus ostralegus Charadriidae
Rubus septentrionalis Rosaceae
Blech